# Make all necessary imports

In [103]:
import pandas as pd
import os
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Join the train and test dataset in to one dataset

The goal here is to make it possible to split the Dataset into train, test and validation sets later on.

In [104]:
train_data = pd.read_csv('../data/train_data.csv')
test_data = pd.read_csv('../data/test_data.csv')

train_data.head()

,ID,Gender,Has a car,Has a property,Children count,Income,Employment status,Education level,Marital status,Dwelling,Age,Employment length,Has a mobile phone,Has a work phone,Has a phone,Has an email,Job title,Family member count,Account age,Is high risk
0,5037048,M,Y,Y,0,135000.0,Working,Secondary / secondary special,Married,With parents,-16271,-3111,1,0,0,0,Core staff,2.0,-17.0,0
1,5044630,F,Y,N,1,135000.0,Commercial associate,Higher education,Single / not married,House / apartment,-10130,-1651,1,0,0,0,Accountants,2.0,-1.0,0
2,5079079,F,N,Y,2,180000.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-12821,-5657,1,0,0,0,Laborers,4.0,-38.0,0
3,5112872,F,Y,Y,0,360000.0,Commercial associate,Higher education,Single / not married,House / apartment,-20929,-2046,1,0,0,1,Managers,1.0,-11.0,0
4,5105858,F,N,N,0,270000.0,Working,Secondary / secondary special,Separated,House / apartment,-16207,-515,1,0,1,0,NaN,1.0,-41.0,0


In [105]:
combined_data = pd.concat([train_data, test_data], axis=0)
combined_data = combined_data.reset_index(drop=True)
combined_data.head()

,ID,Gender,Has a car,Has a property,Children count,Income,Employment status,Education level,Marital status,Dwelling,Age,Employment length,Has a mobile phone,Has a work phone,Has a phone,Has an email,Job title,Family member count,Account age,Is high risk
0,5037048,M,Y,Y,0,135000.0,Working,Secondary / secondary special,Married,With parents,-16271,-3111,1,0,0,0,Core staff,2.0,-17.0,0
1,5044630,F,Y,N,1,135000.0,Commercial associate,Higher education,Single / not married,House / apartment,-10130,-1651,1,0,0,0,Accountants,2.0,-1.0,0
2,5079079,F,N,Y,2,180000.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-12821,-5657,1,0,0,0,Laborers,4.0,-38.0,0
3,5112872,F,Y,Y,0,360000.0,Commercial associate,Higher education,Single / not married,House / apartment,-20929,-2046,1,0,0,1,Managers,1.0,-11.0,0
4,5105858,F,N,N,0,270000.0,Working,Secondary / secondary special,Separated,House / apartment,-16207,-515,1,0,1,0,NaN,1.0,-41.0,0


In [106]:
train_data.shape, test_data.shape, combined_data.shape

((29165, 20), (7292, 20), (36457, 20))

# Look through the joined dataset to find missing values and other outliers

In [107]:
combined_data.columns

Index(['ID', 'Gender', 'Has a car', 'Has a property', 'Children count',
       'Income', 'Employment status', 'Education level', 'Marital status',
       'Dwelling', 'Age', 'Employment length', 'Has a mobile phone',
       'Has a work phone', 'Has a phone', 'Has an email', 'Job title',
       'Family member count', 'Account age', 'Is high risk'],
      dtype='object')

In [108]:
combined_data.isnull().sum()

ID                         0
Gender                     0
Has a car                  0
Has a property             0
Children count             0
Income                     0
Employment status          0
Education level            0
Marital status             0
Dwelling                   0
Age                        0
Employment length          0
Has a mobile phone         0
Has a work phone           0
Has a phone                0
Has an email               0
Job title              11323
Family member count        0
Account age                0
Is high risk               0
dtype: int64

We can see that there are missing values in the ```Job title``` column. I decided to fill this missing values with Unknown.

In [109]:
# replace null values with unknown
combined_data['Job title'].fillna('Unknown', inplace=True)
combined_data.isnull().sum()

/var/folders/yw/h7pq8m2916x9_nlnpf_t3_jw0000gn/T/ipykernel_62288/2787563492.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_data['Job title'].fillna('Unknown', inplace=True)


ID                     0
Gender                 0
Has a car              0
Has a property         0
Children count         0
Income                 0
Employment status      0
Education level        0
Marital status         0
Dwelling               0
Age                    0
Employment length      0
Has a mobile phone     0
Has a work phone       0
Has a phone            0
Has an email           0
Job title              0
Family member count    0
Account age            0
Is high risk           0
dtype: int64

In [110]:
combined_data['Job title'].value_counts()

Job title
Unknown                  11323
Laborers                  6211
Core staff                3591
Sales staff               3485
Managers                  3012
Drivers                   2138
High skill tech staff     1383
Accountants               1241
Medicine staff            1207
Cooking staff              655
Security staff             592
Cleaning staff             551
Private service staff      344
Low-skill Laborers         175
Waiters/barmen staff       174
Secretaries                151
HR staff                    85
Realty agents               79
IT staff                    60
Name: count, dtype: int64

## Now we can look through the different columns of the dataset to find the datatypes of the columns

In [111]:
print(combined_data["Gender"].value_counts(), "\n")
print(combined_data["Has a car"].value_counts(), "\n")
print(combined_data["Has a property"].value_counts(), "\n")
print(combined_data["Children count"].value_counts(), "\n")
print(combined_data["Income"].value_counts(), "\n")
print(combined_data["Employment status"].value_counts(), "\n")
print(combined_data["Education level"].value_counts(), "\n")
print(combined_data["Marital status"].value_counts(), "\n")
print(combined_data["Dwelling"].value_counts(), "\n")
print(combined_data["Age"].value_counts(), "\n")
print(combined_data["Employment length"].value_counts(), "\n")
print(combined_data["Has a mobile phone"].value_counts(), "\n")
print(combined_data["Has a work phone"].value_counts(), "\n")
print(combined_data["Has a phone"].value_counts(), "\n")
print(combined_data["Has an email"].value_counts(), "\n")
print(combined_data["Job title"].value_counts(), "\n")
print(combined_data["Family member count"].value_counts(), "\n")
print(combined_data["Account age"].value_counts(), "\n")
print(combined_data["Is high risk"].value_counts(), "\n")

Gender
F    24430
M    12027
Name: count, dtype: int64 

Has a car
N    22614
Y    13843
Name: count, dtype: int64 

Has a property
Y    24506
N    11951
Name: count, dtype: int64 

Children count
0     25201
1      7492
2      3256
3       419
4        63
5        20
14        3
7         2
19        1
Name: count, dtype: int64 

Income
135000.0    4309
180000.0    3097
157500.0    3089
112500.0    2956
225000.0    2926
            ... 
164250.0       1
432000.0       1
177750.0       1
531000.0       1
62100.0        1
Name: count, Length: 265, dtype: int64 

Employment status
Working                 18819
Commercial associate     8490
Pensioner                6152
State servant            2985
Student                    11
Name: count, dtype: int64 

Education level
Secondary / secondary special    24777
Higher education                  9864
Incomplete higher                 1410
Lower secondary                    374
Academic degree                     32
Name: count, dtype: int64

Based on this findigns we can now decide on how to handle the different columns in the dataset. I decided to use one hot encoding for the categorical columns (```Job title```, ```Employment status```, ```Èducation level```, ```Marital status``` and ```Dwelling```) and to just map the binary columns to numerical values (```Gender```, ```Has a car``` and ```Has a property```)

In [112]:
def data_preprocessing(data):
    data = data.drop(columns=["ID", "Has a mobile phone"])
    data = data.dropna()
    data = data.reset_index(drop=True)
    data = pd.get_dummies(data, columns=["Job title", "Employment status", "Education level", "Marital status", "Dwelling"], drop_first=True)
    data["Gender"] = data["Gender"].map({"M": 0, "F": 1})
    data["Has a car"] = data["Has a car"].map({"N": 0, "Y": 1})
    data["Has a property"] = data["Has a property"].map({"N": 0, "Y": 1})
    bool_columns = data.select_dtypes(include='bool').columns
    data[bool_columns] = data[bool_columns].astype(int)
    return data

Now we can apply this '''data_preprocessing''' function to the ```combined_data``` to get the final dataset that we will use for the model training.

In [113]:
data = data_preprocessing(combined_data)
data.shape

(36457, 48)

## Balancing the dataset

Now we take a look if this dataset is balanced or not. If it is not balanced we can use SMOTE to balance the dataset.

In [114]:
data["Is high risk"].value_counts()

Is high risk
0    35841
1      616
Name: count, dtype: int64

We can see that the dataset is extremly unbalanced. We can use SMOTE to balance the dataset. Therefore we first have to split the dataset into train, test and validation sets. We do this to ensure the data is not leaked from the test and validation set into the training set. This way we can be sure that the test dataset only contains unseen data.

In [115]:
train, test = train_test_split(data, test_size=0.3, stratify=data["Is high risk"], random_state=42)
test, val = train_test_split(test, test_size=0.5, stratify=test["Is high risk"], random_state=42)

In [116]:
train["Is high risk"].value_counts(), test["Is high risk"].value_counts(), val["Is high risk"].value_counts()

(Is high risk
 0    25088
 1      431
 Name: count, dtype: int64,
 Is high risk
 0    5377
 1      92
 Name: count, dtype: int64,
 Is high risk
 0    5376
 1      93
 Name: count, dtype: int64)

We only use SMOTE for the training and the validation dataset. We do this because this way we can train the model on a balanced dataset and test if the model is able to generalize to unseen data that contains the distribution of the original dataset.

In [117]:
X_train = train.drop("Is high risk", axis=1)
Y_train = train["Is high risk"]

X_val = val.drop("Is high risk", axis=1)
Y_val = val["Is high risk"]

In [118]:
smote = SMOTE(random_state=42)
X_train, Y_train = smote.fit_resample(X_train, Y_train)
X_val, Y_val = smote.fit_resample(X_val, Y_val)

In [119]:
train = pd.concat([X_train, Y_train], axis=1)
val = pd.concat([X_val, Y_val], axis=1)

train["Is high risk"].value_counts(), val["Is high risk"].value_counts()

(Is high risk
 0    25088
 1    25088
 Name: count, dtype: int64,
 Is high risk
 0    5376
 1    5376
 Name: count, dtype: int64)

Now we can save the different datasets to csv files to use them in the model training.

In [120]:
if not os.path.exists('../data/preprocessed'):
    os.makedirs('../data/preprocessed')

if os.path.exists('../data/preprocessed/all_data.csv'):
    os.remove('../data/preprocessed/all_data.csv')
if os.path.exists('../data/preprocessed/test_data.csv'):
    os.remove('../data/preprocessed/test_data.csv')
if os.path.exists('../data/preprocessed/val_data.csv'):
    os.remove('../data/preprocessed/val_data.csv')
if os.path.exists('../data/preprocessed/train_data.csv'):
    os.remove('../data/preprocessed/train_data.csv')

data.to_csv('../data/preprocessed/all_data.csv', index=False)
test.to_csv('../data/preprocessed/test_data.csv', index=False)
val.to_csv('../data/preprocessed/val_data.csv', index=False)
train.to_csv('../data/preprocessed/train_data.csv', index=False)